In [ ]:
"""
Note: When running this cell it will prompt you to add a file named kaggle.json in order to proceed and download
the dataset directly from kaggle. Just make a kaggle account, go to account settings, then click "Create New Token".
It will download a file called kaggle.json which you will add to the prompt below. The dataset will then install.
"""
!pip install -q kaggle

# Upload the kaggle.json file
from google.colab import files
files.upload()  # This will prompt you to upload your kaggle.json file

# Move kaggle.json to the correct directory
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Set permissions to secure the API key

# Download the SisFall dataset using the Kaggle API
!kaggle datasets download -d nvnikhil0001/sis-fall-original-dataset -p /content;

# Unzip the dataset into a folder named 'sis-fall-original-dataset' in the Colab environment
!unzip /content/sis-fall-original-dataset.zip -d /content/sis-fall-original-dataset;



In [ ]:
#Contents of dataset
!ls /content/sis-fall-original-dataset/SisFall_dataset/

Readme.txt  SA04  SA08	SA12  SA16  SA20  SE01	SE05  SE09  SE13
SA01	    SA05  SA09	SA13  SA17  SA21  SE02	SE06  SE10  SE14
SA02	    SA06  SA10	SA14  SA18  SA22  SE03	SE07  SE11  SE15
SA03	    SA07  SA11	SA15  SA19  SA23  SE04	SE08  SE12  Supplementary.pdf


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.patches as patches
import os
import math

In [ ]:
#Path to dataset
path = '/content/sis-fall-original-dataset/SisFall_dataset/'

l = os.listdir(path)  #Get the list of all directories and files in the dataset
l.sort()


f = []  #List for paths starting with 'F'
d = []  #List for paths starting with 'D'


for sub_fol in l[1:39]:  #Range to exclude .txt and .pdf file
    subfolder_path = os.path.join(path, sub_fol)  #Build the full path for the subfolder

    #Check if the entry is a directory
    if os.path.isdir(subfolder_path):
        for sub_sub_fol in os.listdir(subfolder_path):  #List all files in the subfolder
            path_ = os.path.join(subfolder_path, sub_sub_fol)  #Construct the full path
            if sub_sub_fol.startswith('F'):  #Check if the file starts with 'F'
                f.append(path_)  #Add to F list
            elif sub_sub_fol.startswith('D'):  #Check if the file starts with 'D'
                d.append(path_)  #Add to D list



In [ ]:
l

['Readme.txt',
 'SA01',
 'SA02',
 'SA03',
 'SA04',
 'SA05',
 'SA06',
 'SA07',
 'SA08',
 'SA09',
 'SA10',
 'SA11',
 'SA12',
 'SA13',
 'SA14',
 'SA15',
 'SA16',
 'SA17',
 'SA18',
 'SA19',
 'SA20',
 'SA21',
 'SA22',
 'SA23',
 'SE01',
 'SE02',
 'SE03',
 'SE04',
 'SE05',
 'SE06',
 'SE07',
 'SE08',
 'SE09',
 'SE10',
 'SE11',
 'SE12',
 'SE13',
 'SE14',
 'SE15',
 'Supplementary.pdf']

In [ ]:
print(len(f))
print(len(d))

1798
2707


In [ ]:
import pandas as pd
import math
from tqdm import tqdm  # Import tqdm for progress tracking
import os  # Import os for working with file paths

# List to collect all DataFrames
dataframes_F = []

# Use tqdm to wrap the iterable for progress tracking
for p, path in enumerate(tqdm(f, desc="Processing Files")):
    with open(path, 'r') as file:
        content = file.read()

    # Clean and process the content
    content = content.replace(' ', '').split(';\n')

    # Split each line by comma and filter out empty lines
    rows = [line.split(',') for line in content if line]

    # Convert to integers and append 1 to each row where 1 indicates fall
    rows = [[int(value) for value in row] + [1] for row in rows if row != ['']]

    # Calculate RMV and add a new column to the end
    rows = [
        row + [math.sqrt(row[0]**2 + row[1]**2 + row[2]**2)]
        for row in rows if row != []
    ]

    # Create a DataFrame for the current file
    df = pd.DataFrame(
        rows,
        columns=[
            'Acceleration in X axis for ADXL345',
            'Acceleration in Y axis for ADXL345',
            'Acceleration in Z axis for ADXL345',
            'Rotation in X axis for ITG3200',
            'Rotation in Y axis for ITG3200',
            'Rotation in Z axis for ITG3200',
            'Acceleration in X axis for MMA8451Q',
            'Acceleration in Y axis for MMA8451Q',
            'Acceleration in Z axis for MMA8451Q',
            'Fall',
            'SMV'
        ]
    )

    # Add a column with just the file name (after the last slash)
    df['Filename'] = os.path.basename(path)

    # Append the DataFrame to the list
    dataframes_F.append(df)

# Combine all DataFrames into one if needed later
# df_combined = pd.concat(dataframes, ignore_index=True)


Processing Files: 100%|██████████| 1798/1798 [03:12<00:00,  9.32it/s]


In [ ]:
dataframes_F[0].head()


,Acceleration in X axis for ADXL345,Acceleration in Y axis for ADXL345,Acceleration in Z axis for ADXL345,Rotation in X axis for ITG3200,Rotation in Y axis for ITG3200,Rotation in Z axis for ITG3200,Acceleration in X axis for MMA8451Q,Acceleration in Y axis for MMA8451Q,Acceleration in Z axis for MMA8451Q,Fall,SMV,Filename
0,-96,-381,-74,-706,165,245,-451,-1473,-158,1,399.816208,F01_SA01_R04.txt
1,-112,-390,-59,-835,119,336,-488,-1504,-77,1,410.030487,F01_SA01_R04.txt
2,-108,-399,-34,-884,65,433,-496,-1520,12,1,414.754144,F01_SA01_R04.txt
3,-109,-402,-13,-846,-6,533,-480,-1526,95,1,416.718130,F01_SA01_R04.txt
4,-93,-401,2,-728,-95,630,-430,-1519,148,1,411.647908,F01_SA01_R04.txt


In [ ]:
import pandas as pd
import math
from tqdm import tqdm  # Import tqdm for progress tracking
import os  # Import os for working with file paths

# Define the bit conversion function
def bit_conversion(raw_adc_value, max_range, bit_resolution):
    """
    Converts raw ADC value to a floating point value based on the range and bit resolution.

    :param raw_adc_value: Raw ADC value from the sensor
    :param max_range: The maximum range of the sensor (e.g., ±16g, ±8g, etc.)
    :param bit_resolution: The bit resolution of the ADC (e.g., 13, 14, 16)

    :return: Converted floating point value
    """
    # Calculate the maximum ADC value based on the bit resolution
    max_adc_value = 2**bit_resolution - 1

    # Convert the raw ADC value to a floating-point value
    converted_value = (raw_adc_value / max_adc_value) * max_range

    return converted_value

# List to collect all DataFrames
dataframes_F = []

# Use tqdm to wrap the iterable for progress tracking
for p, path in enumerate(tqdm(f, desc="Processing Files")):
    with open(path, 'r') as file:
        content = file.read()

    # Clean and process the content
    content = content.replace(' ', '').split(';\n')

    # Split each line by comma and filter out empty lines
    rows = [line.split(',') for line in content if line]

    # Convert to integers and append 1 to each row where 1 indicates fall
    rows = [[int(value) for value in row] + [1] for row in rows if row != ['']]

    # Calculate SMV and add a new column to the end
    rows = [
        row
        + [math.sqrt(row[0]**2 + row[1]**2 + row[2]**2)]  # SMV for ADX345
        + [math.sqrt(row[6]**2 + row[7]**2 + row[8]**2)]  # SMV for MMA8451Q
        for row in rows if row != []
    ]

    # Create a DataFrame for the current file
    df = pd.DataFrame(
        rows,
        columns=[
            'Acceleration in X axis for ADXL345',
            'Acceleration in Y axis for ADXL345',
            'Acceleration in Z axis for ADXL345',
            'Rotation in X axis for ITG3200',
            'Rotation in Y axis for ITG3200',
            'Rotation in Z axis for ITG3200',
            'Acceleration in X axis for MMA8451Q',
            'Acceleration in Y axis for MMA8451Q',
            'Acceleration in Z axis for MMA8451Q',
            'Fall',
            'SMV',
            'SMV_MMA',
        ]
    )

    # Add a column with just the file name (after the last slash)
    df['Filename'] = os.path.basename(path)

    # Apply bit conversion to the respective columns
    df["Acceleration in X axis for ADXL345"] = df["Acceleration in X axis for ADXL345"].apply(bit_conversion, args=(16, 13))
    df["Acceleration in Y axis for ADXL345"] = df["Acceleration in Y axis for ADXL345"].apply(bit_conversion, args=(16, 13))
    df["Acceleration in Z axis for ADXL345"] = df["Acceleration in Z axis for ADXL345"].apply(bit_conversion, args=(16, 13))

    df["Rotation in X axis for ITG3200"] = df["Rotation in X axis for ITG3200"].apply(bit_conversion, args=(2000, 16))
    df["Rotation in Y axis for ITG3200"] = df["Rotation in Y axis for ITG3200"].apply(bit_conversion, args=(2000, 16))
    df["Rotation in Z axis for ITG3200"] = df["Rotation in Z axis for ITG3200"].apply(bit_conversion, args=(2000, 16))

    df["Acceleration in X axis for MMA8451Q"] = df["Acceleration in X axis for MMA8451Q"].apply(bit_conversion, args=(8, 14))
    df["Acceleration in Y axis for MMA8451Q"] = df["Acceleration in Y axis for MMA8451Q"].apply(bit_conversion, args=(8, 14))
    df["Acceleration in Z axis for MMA8451Q"] = df["Acceleration in Z axis for MMA8451Q"].apply(bit_conversion, args=(8, 14))

    # Append the DataFrame to the list
    dataframes_F.append(df)

# Combine all DataFrames into one if needed later
# df_combined = pd.concat(dataframes, ignore_index=True)


import pandas as pd
import math
from tqdm import tqdm  # Import tqdm for progress tracking
import os  # Import os for working with file paths


# List to collect all DataFrames
dataframes_D = []

# Use tqdm to wrap the iterable for progress tracking
for p, path in enumerate(tqdm(d, desc="Processing Files")):
    with open(path, 'r') as file:
        content = file.read()

    # Clean and process the content
    content = content.replace(' ', '').split(';\n')

    # Split each line by comma and filter out empty lines
    rows = [line.split(',') for line in content if line]

    # Convert to integers and append 1 to each row where 1 indicates fall
    rows = [[int(value) for value in row] + [1] for row in rows if row != ['']]

    # Calculate SMV and add a new column to the end
    rows = [
        row
        + [math.sqrt(row[0]**2 + row[1]**2 + row[2]**2)]  # SMV for ADX345
        + [math.sqrt(row[6]**2 + row[7]**2 + row[8]**2)]  # SMV for MMA8451Q
        for row in rows if row != []
    ]

    # Create a DataFrame for the current file
    df = pd.DataFrame(
        rows,
        columns=[
            'Acceleration in X axis for ADXL345',
            'Acceleration in Y axis for ADXL345',
            'Acceleration in Z axis for ADXL345',
            'Rotation in X axis for ITG3200',
            'Rotation in Y axis for ITG3200',
            'Rotation in Z axis for ITG3200',
            'Acceleration in X axis for MMA8451Q',
            'Acceleration in Y axis for MMA8451Q',
            'Acceleration in Z axis for MMA8451Q',
            'Fall',
            'SMV',
            'SMV_MMA'
        ]
    )

    # Add a column with just the file name (after the last slash)
    df['Filename'] = os.path.basename(path)

    # Apply bit conversion to the respective columns
    df["Acceleration in X axis for ADXL345"] = df["Acceleration in X axis for ADXL345"].apply(bit_conversion, args=(16, 13))
    df["Acceleration in Y axis for ADXL345"] = df["Acceleration in Y axis for ADXL345"].apply(bit_conversion, args=(16, 13))
    df["Acceleration in Z axis for ADXL345"] = df["Acceleration in Z axis for ADXL345"].apply(bit_conversion, args=(16, 13))

    df["Rotation in X axis for ITG3200"] = df["Rotation in X axis for ITG3200"].apply(bit_conversion, args=(2000, 16))
    df["Rotation in Y axis for ITG3200"] = df["Rotation in Y axis for ITG3200"].apply(bit_conversion, args=(2000, 16))
    df["Rotation in Z axis for ITG3200"] = df["Rotation in Z axis for ITG3200"].apply(bit_conversion, args=(2000, 16))

    df["Acceleration in X axis for MMA8451Q"] = df["Acceleration in X axis for MMA8451Q"].apply(bit_conversion, args=(8, 14))
    df["Acceleration in Y axis for MMA8451Q"] = df["Acceleration in Y axis for MMA8451Q"].apply(bit_conversion, args=(8, 14))
    df["Acceleration in Z axis for MMA8451Q"] = df["Acceleration in Z axis for MMA8451Q"].apply(bit_conversion, args=(8, 14))

    # Append the DataFrame to the list
    dataframes_D.append(df)

# Combine all DataFrames into one if needed later
# df_combined = pd.concat(dataframes, ignore_index=True)


Processing Files: 100%|██████████| 2707/2707 [04:46<00:00,  9.45it/s]


In [ ]:
dataframes_D[0]

,Acceleration in X axis for ADXL345,Acceleration in Y axis for ADXL345,Acceleration in Z axis for ADXL345,Rotation in X axis for ITG3200,Rotation in Y axis for ITG3200,Rotation in Z axis for ITG3200,Acceleration in X axis for MMA8451Q,Acceleration in Y axis for MMA8451Q,Acceleration in Z axis for MMA8451Q,Fall,SMV,SMV_MMA,Filename
0,-0.021487,-0.480527,-0.105482,-0.396735,3.784237,-0.091554,-0.031740,-0.462431,-0.051761,1,252.097203,955.128264,D17_SA01_R05.txt
1,-0.021487,-0.492248,-0.107435,-0.396735,3.784237,-0.122072,-0.032229,-0.462431,-0.055179,1,258.166613,955.998954,D17_SA01_R05.txt
2,-0.027347,-0.486387,-0.107435,-0.488289,3.784237,-0.061036,-0.032717,-0.463896,-0.056156,1,255.385982,959.277853,D17_SA01_R05.txt
3,-0.023440,-0.484434,-0.105482,-0.488289,3.753719,-0.152590,-0.031740,-0.462919,-0.055667,1,254.094471,957.039707,D17_SA01_R05.txt
4,-0.025394,-0.490294,-0.103528,-0.518807,3.723201,-0.122072,-0.030764,-0.464384,-0.055667,1,256.863777,959.878117,D17_SA01_R05.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,-0.009767,-0.488341,-0.103528,-0.274662,2.990768,-0.091554,-0.010743,-0.466825,-0.056156,1,255.605164,963.143292,D17_SA01_R05.txt
4996,-0.007813,-0.488341,-0.113295,-0.274662,3.082322,-0.152590,-0.009278,-0.464872,-0.053714,1,256.670996,958.522300,D17_SA01_R05.txt
4997,-0.005860,-0.490294,-0.097668,-0.274662,3.173877,-0.091554,-0.012208,-0.468290,-0.051761,1,255.949214,965.164235,D17_SA01_R05.txt
4998,-0.003907,-0.492248,-0.103528,-0.244144,3.204395,-0.152590,-0.013184,-0.465849,-0.052249,1,257.520873,960.361390,D17_SA01_R05.txt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# IMU sampling rate
sample_fre=200

# Window Size
winsize=1

# feature length
feature_length=int(winsize*sample_fre)


In [ ]:
features_f=[]
for dataframe_f in dataframes_F:
    SMVs= dataframe_f.loc[:,["SMV"]]

    # Find the index of the largest value
    max_index = SMVs.idxmax()

    # Get the actual integer index value
    max_index_value = max_index.iloc[0] #get the integer value of the index

    # Calculate start and end indices for the range
    start_index = max_index_value - int(feature_length/2)
    end_index =  max_index_value + int(feature_length/2)

    if end_index > len(dataframe_f) or start_index<0:
      break

    # Extract the range of rows
    centered_data = dataframe_f.loc[start_index:end_index, ["Acceleration in X axis for ADXL345","Acceleration in Y axis for ADXL345","Acceleration in Z axis for ADXL345","Filename"]]  # Extract rows using .loc

    # contact the data (xyz->xyz->xyz)

    # Reshape the extracted data and append to the features list
    feature_per_txt = centered_data[
        ["Acceleration in X axis for ADXL345", "Acceleration in Y axis for ADXL345", "Acceleration in Z axis for ADXL345"]
        ].values.reshape(1, -1)  # Reshape to (1, feature_length * 3)
    # features_f.append((feature_per_txt, 1,int(centered_data["Filename"].iloc[0][1:3])))  # Store feature and filename
    features_f.append((feature_per_txt, 1))

    # Now, 'features' contains a list of tuples: (feature_array, filename)

    # feature_per_txt=[]
    # for i in range(len(centered_data)):
    #     feature_per_txt.extend(centered_data.iloc[i,:3])


In [ ]:
len(features_f)

In [ ]:
import random
features_d=[]
for dataframe_f in dataframes_F:

    # Get the total number of rows in the dataframe
    total_rows = len(dataframe_f)

    # Calculate a random start index within the allowed range
    start_index = random.randint(0, total_rows - feature_length)

    # Calculate the end index based on feature length
    end_index = start_index + feature_length

    # Extract the random range of rows
    centered_data = dataframe_f.loc[
        start_index:end_index,
        ["Acceleration in X axis for ADXL345", "Acceleration in Y axis for ADXL345", "Acceleration in Z axis for ADXL345", "Filename"]
    ]

    # contact the data (xyz->xyz->xyz)
    # Reshape the extracted data and append to the features list
    feature_per_txt = centered_data[
        ["Acceleration in X axis for ADXL345", "Acceleration in Y axis for ADXL345", "Acceleration in Z axis for ADXL345"]
        ].values.reshape(1, -1)  # Reshape to (1, feature_length * 3)
    # features_d.append((feature_per_txt, 0,int(centered_data["Filename"].iloc[0][1:3])))  # Store feature and filename
    features_d.append((feature_per_txt, 0))


In [ ]:
#Code for validation on second IMU:
#********************************


features_f_validate_MMA=[]
for dataframe_f in dataframes_F:
    SMVs= dataframe_f.loc[:,["SMV_MMA"]]

    # Find the index of the largest value
    max_index = SMVs.idxmax()

    # Get the actual integer index value
    max_index_value = max_index.iloc[0] #get the integer value of the index

    # Calculate start and end indices for the range
    start_index = max_index_value - int(feature_length/2)
    end_index =  max_index_value + int(feature_length/2)

    if end_index > len(dataframe_f) or start_index<0:
      break

    # Extract the range of rows
    centered_data = dataframe_f.loc[start_index:end_index, ["Acceleration in X axis for MMA8451Q","Acceleration in Y axis for MMA8451Q","Acceleration in Z axis for MMA8451Q","Filename"]]  # Extract rows using .loc

    # contact the data (xyz->xyz->xyz)

    # Reshape the extracted data and append to the features list
    feature_per_txt = centered_data[
        ["Acceleration in X axis for MMA8451Q", "Acceleration in Y axis for MMA8451Q", "Acceleration in Z axis for MMA8451Q"]
        ].values.reshape(1, -1)  # Reshape to (1, feature_length * 3)
    # features_f.append((feature_per_txt, 1,int(centered_data["Filename"].iloc[0][1:3])))  # Store feature and filename
    features_f_validate_MMA.append((feature_per_txt, 1))

    # Now, 'features' contains a list of tuples: (feature_array, filename)

    # feature_per_txt=[]
    # for i in range(len(centered_data)):
    #     feature_per_txt.extend(centered_data.iloc[i,:3])

In [ ]:
import random
features_d_validate_MMA=[]
for dataframe_f in dataframes_F:

    # Get the total number of rows in the dataframe
    total_rows = len(dataframe_f)

    # Calculate a random start index within the allowed range
    start_index = random.randint(0, total_rows - feature_length)

    # Calculate the end index based on feature length
    end_index = start_index + feature_length

    # Extract the random range of rows
    centered_data = dataframe_f.loc[
        start_index:end_index,
        ["Acceleration in X axis for ADXL345", "Acceleration in Y axis for ADXL345", "Acceleration in Z axis for ADXL345", "Filename"]
    ]

    # contact the data (xyz->xyz->xyz)
    # Reshape the extracted data and append to the features list
    feature_per_txt = centered_data[
        ["Acceleration in X axis for ADXL345", "Acceleration in Y axis for ADXL345", "Acceleration in Z axis for ADXL345"]
        ].values.reshape(1, -1)  # Reshape to (1, feature_length * 3)
    # features_d.append((feature_per_txt, 0,int(centered_data["Filename"].iloc[0][1:3])))  # Store feature and filename
    features_d_validate_MMA.append((feature_per_txt, 0))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import sklearn.metrics as metrics
from torch.utils.data import DataLoader, TensorDataset


# Define the CNN architecture
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()

        # Convolutional feature extraction layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(1, 5), stride=(1, 2), padding=(0, 2))
        self.norm1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(kernel_size=(1, 5), stride=(1, 1))

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(1, 5), stride=(1, 2), padding=(0, 2))
        self.norm2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(kernel_size=(1, 5), stride=(1, 1))

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1, 5), stride=(1, 2), padding=(0, 2))
        self.norm3 = nn.BatchNorm2d(64)
        self.pool3 = nn.MaxPool2d(kernel_size=(1, 5), stride=(1, 1))

        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(1, 5), stride=(1, 2), padding=(0, 2))
        self.norm4 = nn.BatchNorm2d(128)
        self.pool4 = nn.MaxPool2d(kernel_size=(1, 5), stride=(1, 1))

        # Fully connected layers for classification
        self.fc1 = nn.Linear(3968, 128)
        self.fc2 = nn.Linear(128, 2)  # Assuming 10 classes for classification

    def forward(self, x):
        x = self.pool1(F.relu(self.norm1(self.conv1(x))))
        x = self.pool2(F.relu(self.norm2(self.conv2(x))))
        x = self.pool3(F.relu(self.norm3(self.conv3(x))))
        x = self.pool4(F.relu(self.norm4(self.conv4(x))))

        # Flatten for fully connected layers
        x = torch.flatten(x, 1)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.softmax(x, dim=1)



def train_model(model, criterion, optimizer, num_epochs, val_data,scheduler):

    best_val_loss = float('inf')  # Initialize with a very large value
    patience = 3  # Number of epochs to wait before stopping
    epochs_without_improvement = 0


    for epoch in range(num_epochs):
        train_loss = 0.0  # Accumulate training loss
        num_batches = 0

        for inputs_train, labels_train in train_loader:

            # Zero the parameter gradients
            optimizer.zero_grad()

             # Reshape inputs_train to add a channel dimension
            inputs_train = inputs_train.unsqueeze(1)
            inputs_train = inputs_train.unsqueeze(-2)   # Add channel dimension
            # print(inputs_train.shape)

            #(Batch Size, Channels, Height, Width)
            # 当前的特征是 [32,604]这表示你有 32 个样本（batch size），
            # 每个样本有 604 维特征，但这并不符合 nn.Conv2d 的输入要求。
            #为了正确使用 nn.Conv2d，你需要将你的输入调整为四维张量。
            #(Batch Size, Channels, Height, Width)=(32,1,1,604)
            #这表示每个样本有一个通道，高度为 1，宽度为 604。


            # Forward pass
            outputs_train = model(inputs_train)

            # Calculate loss
            loss_train = criterion(outputs_train, labels_train)
            train_loss += loss_train.item()  # Accumulate loss

            # Backward pass
            loss_train.backward()
            optimizer.step()

        # Calculate average training loss for the epoch
        avg_train_loss = train_loss / len(train_loader)

        # Calculate validation loss for the epoch
        val_loss = 0.0
        with torch.no_grad():
            for inputs_val, labels_val in val_loader:
                inputs_val = inputs_val.unsqueeze(1)
                inputs_val = inputs_val.unsqueeze(-2)
                outputs_val = model(inputs_val)
                loss_val = criterion(outputs_val, labels_val)
                val_loss += loss_val.item()

        avg_val_loss = val_loss / len(val_loader)

        # Adjust learning rate using the scheduler
        scheduler.step(avg_val_loss)

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss_train: {avg_train_loss:.4f}, Loss_val: {avg_val_loss:.4f}")

        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
              print("Early stopping triggered.")

              break


#The error arises because the input to nn.Conv2d expects a 4-dimensional
#tensor in the format: [batch_size, channels, height, width].
#The code adds a channel dimension by calling inputs_train = inputs_train.unsqueeze(1)
#but this results in a tensor with dimensions [batch_size, channels, feature_length]. The missing dimension is the height which is required by nn.Conv2d and nn.BatchNorm2d.


In [ ]:
# Training function with confusion matrix calculation
from sklearn.metrics import confusion_matrix, classification_report
def evaluate_model(model, loader):
    """Evaluate the model and compute the confusion matrix."""
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in loader:
            # Add channel and spatial dimensions
            inputs = inputs.unsqueeze(1).unsqueeze(-2)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            # 参数 1 (outputs): 这是模型的输出张量。
            # 参数 2 (dim=1): 指定沿哪个维度找最大值。
            # dim=1 表示在每一行中找最大值（即在类别维度上）。
            # 每行对应一个样本，每列对应一个类别。

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())


    # Calculate confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    # print("Confusion Matrix:",precision=4)
    # print(cm)

    # # Optional: Display additional metrics
    # print("\nClassification Report:")
    # print(classification_report(all_labels, all_preds))

    # Format the confusion matrix with desired precision using numpy
    formatted_cm = np.array2string(cm, precision=4)  # Format with precision=4

    print("Confusion Matrix:")
    print(formatted_cm)  # Print the formatted matrix

    # Optional: Display additional metrics
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds))


    return cm

In [ ]:
# get 60 % training data, 20% validation data and 20% test data
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import sklearn.metrics as metrics
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import random_split


features = features_f + features_d
random.shuffle(features)

train_size = int(0.6 * len(features))
val_size = int(0.2 * len(features))
test_size = len(features) - train_size - val_size

train_data = features[:train_size]
val_data = features[train_size:train_size+val_size]
test_data = features[train_size+val_size:]


# Define batch size
batch_size =32

# Create a dataset and data loader for training
train_inputs = [data[0] for data in train_data]
train_labels = [data[1] for data in train_data]

train_inputs = torch.tensor(np.vstack(train_inputs), dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.long)

# Create a DataLoader for training data
train_dataset = TensorDataset(train_inputs, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


# Separate inputs and labels from val_data
val_inputs = [data[0] for data in val_data]
val_labels = [data[1] for data in val_data]

# Convert to tensors
val_inputs = torch.tensor(np.vstack(val_inputs), dtype=torch.float32)  # Stack and convert to tensor
val_labels = torch.tensor(val_labels, dtype=torch.long)

val_dataset = TensorDataset(val_inputs, val_labels)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# for test data
test_inputs = [data[0] for data in test_data]
test_labels = [data[1] for data in test_data]

test_inputs = torch.tensor(np.vstack(test_inputs), dtype=torch.float32)
test_labels = torch.tensor(test_labels, dtype=torch.long)

test_dataset = TensorDataset(test_inputs, test_labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)




In [ ]:
# get 80 % training data,20% test data and another IMU for validation
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import sklearn.metrics as metrics
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import random_split


features = features_f + features_d
features = features_f_validate_MMA+features

random.shuffle(features)

train_size = int(0.8 * len(features))
val_size = int(0.2 * len(features))
test_size = len(features) - train_size - val_size

train_data = features[:train_size]
val_data = features[train_size:train_size+val_size]
test_data = features[train_size+val_size:]


# Define batch size
batch_size =32

# Create a dataset and data loader for training
train_inputs = [data[0] for data in train_data]
train_labels = [data[1] for data in train_data]

train_inputs = torch.tensor(np.vstack(train_inputs), dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.long)

# Create a DataLoader for training data
train_dataset = TensorDataset(train_inputs, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


# Separate inputs and labels from val_data
val_inputs = [data[0] for data in val_data]
val_labels = [data[1] for data in val_data]

# Convert to tensors
val_inputs = torch.tensor(np.vstack(val_inputs), dtype=torch.float32)  # Stack and convert to tensor
val_labels = torch.tensor(val_labels, dtype=torch.long)

val_dataset = TensorDataset(val_inputs, val_labels)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# for test data
test_inputs = [data[0] for data in test_data]
test_labels = [data[1] for data in test_data]

test_inputs = torch.tensor(np.vstack(test_inputs), dtype=torch.float32)
test_labels = torch.tensor(test_labels, dtype=torch.long)

test_dataset = TensorDataset(test_inputs, test_labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)




In [ ]:
model=CustomCNN()
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()

model.train()
# SGD optimizer with momentum
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Adaptive learning rate scheduler (ReduceLROnPlateau)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

num_epochs = 50

train_model(model, criterion, optimizer, num_epochs, val_data,scheduler)

evaluate_model(model, val_loader)

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/50], Loss_train: 0.5049, Loss_val: 0.4085
Epoch [2/50], Loss_train: 0.3785, Loss_val: 0.3647
Epoch [3/50], Loss_train: 0.3516, Loss_val: 0.3527
Epoch [4/50], Loss_train: 0.3431, Loss_val: 0.3468
Epoch [5/50], Loss_train: 0.3382, Loss_val: 0.3447
Epoch [6/50], Loss_train: 0.3359, Loss_val: 0.3422
Epoch [7/50], Loss_train: 0.3330, Loss_val: 0.3404
Epoch [8/50], Loss_train: 0.3329, Loss_val: 0.3390
Epoch [9/50], Loss_train: 0.3309, Loss_val: 0.3384
Epoch [10/50], Loss_train: 0.3307, Loss_val: 0.3380
Epoch [11/50], Loss_train: 0.3296, Loss_val: 0.3370
Epoch [12/50], Loss_train: 0.3293, Loss_val: 0.3365
Epoch [13/50], Loss_train: 0.3307, Loss_val: 0.3361
Epoch [14/50], Loss_train: 0.3278, Loss_val: 0.3349
Epoch [15/50], Loss_train: 0.3267, Loss_val: 0.3349
Epoch [16/50], Loss_train: 0.3265, Loss_val: 0.3345
Epoch [17/50], Loss_train: 0.3264, Loss_val: 0.3344
Epoch [18/50], Loss_train: 0.3273, Loss_val: 0.3341
Epoch [19/50], Loss_train: 0.3254, Loss_val: 0.3340
Epoch [20/50], Loss_t

array([[336,   9],
       [  4, 370]])

In [ ]:
evaluate_model(model, test_loader)
evaluate_model(model, val_loader)
evaluate_model(model, train_loader)

Confusion Matrix:
[[356   4]
 [  2 358]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       360
           1       0.99      0.99      0.99       360

    accuracy                           0.99       720
   macro avg       0.99      0.99      0.99       720
weighted avg       0.99      0.99      0.99       720

Confusion Matrix:
[[336   9]
 [  4 370]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       345
           1       0.98      0.99      0.98       374

    accuracy                           0.98       719
   macro avg       0.98      0.98      0.98       719
weighted avg       0.98      0.98      0.98       719

Confusion Matrix:
[[1075   18]
 [   2 1062]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1093
           1       0.98      1.00     

array([[1075,   18],
       [   2, 1062]])